In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201203-1613-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [14]:
mt=hl.read_matrix_table(f'{temp_dir}/intervalwgs/WGS_final_march_2020_dbsnp_v53.mt')

In [21]:
mt.filter_rows((mt.locus.contig=="chrX") & (mt.locus.position==1452779)).rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:1452779,"[""GA"",""G""]","""rs778776897"""


In [10]:
dbsnp_vcf = f"{temp_dir}/intervalwgs/dbsnp_v53/dbsnp_153.hg38.vcf.gz"
dbsnp_mt = hl.import_vcf(dbsnp_vcf, force_bgz=True, reference_genome='GRCh38',
                         skip_invalid_loci=True)
dbsnp_mt = dbsnp_mt.key_rows_by(
    'locus').distinct_by_row().key_rows_by('locus', 'alleles')

In [2]:
dbsnp_v53=hl.read_matrix_table(f'{temp_dir}/intervalwgs/dbsnp_v53/dbsnp_v53_split.mt')

In [3]:
dbsnp_v53.filter_rows((dbsnp_v53.locus.contig=="chrX") & (dbsnp_v53.locus.position==1452780)).rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str


In [5]:
dbsnp_v53.filter_rows((dbsnp_v53.locus.contig=="chrX") & (dbsnp_v53.rsid=='rs778776897')).rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str


In [12]:
dbsnp_v53.filter_rows((dbsnp_v53.locus.contig=="chrX") & (dbsnp_v53.locus.position==1452779)).rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:1452779,"[""G"",""A""]","""rs1440661997"""


In [11]:
dbsnp_mt.filter_rows((dbsnp_mt.locus.contig=="chrX") & (dbsnp_mt.rsid=='rs778776897')).rsid.show()

2020-12-03 16:40:38 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-12-03 16:44:31 Hail: INFO: Coerced sorted dataset
2020-12-03 16:44:31 Hail: INFO: reading 5 of 106 data partitions


locus,alleles,rsid
locus<GRCh38>,array<str>,str


In [3]:
mt.count()

(116382870, 11863)

In [2]:
dbsnp_vcf = "s3a://intervalwgs-qc/dbsnp_153.hg38.vcf.gz"
dbsnp_mt = hl.import_vcf(dbsnp_vcf, force_bgz=True, reference_genome='GRCh38',
                         skip_invalid_loci=True)

In [3]:
mt.filter_rows((mt.locus.contig=="chrX") & (mt.locus.position==1452779)).rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:1452779,"[""GA"",""G""]","""rs778776897"""


In [6]:
chrX_mt=hl.read_matrix_table( 'hdfs://spark-master:9820//chrX_dbsnp_v153_split.mt')

In [9]:
sm = hl.split_multi(chrX_mt)

In [10]:
sm.filter_rows(sm.locus.position==1452779).rsid.show()

2020-10-26 13:29:54 Hail: INFO: reading 1 of 900 data partitions


locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:1452779,"[""GA"",""G""]",NA


In [7]:
chrX_mt =chrX_mt.key_rows_by('locus').distinct_by_row().key_rows_by('locus', 'alleles')


In [8]:
chrX_mt.filter_rows(chrX_mt.locus.position==1452779).rsid.show()

2020-10-26 13:18:24 Hail: INFO: Coerced sorted dataset
2020-10-26 13:18:24 Hail: INFO: reading 1 of 896 data partitions


locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:1452779,"[""GA"",""G""]",NA


In [ ]:
dbsnp_v53orig=hl.read_matrix_table(f'{temp_dir}/intervalwgs/dbsnp_v53/dbsnp_v53.mt')


In [7]:
dbsnp_v53=hl.read_matrix_table(f'{temp_dir}/intervalwgs/dbsnp_v53/dbsnp_v53_split.mt')


In [6]:
dbsnp_v53.filter_rows(dbsnp_v53.rsid=='rs778776897').show()

KeyboardInterrupt: 

In [8]:
dbsnp_v53.filter_rows(dbsnp_v53.locus.position==1452780).rsid.show()

KeyboardInterrupt: 

In [13]:
hl.export_vcf(dbsnp_v53,f'{tmp_dir}/dbsnp_v53_split.vcf.gz')

2020-10-21 14:40:49 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
2020-10-21 14:42:32 Hail: INFO: merging 212 files totalling 12.3G...
2020-10-21 14:43:05 Hail: INFO: while writing:
    hdfs://spark-master:9820//dbsnp_v53_split.vcf.gz
  merge time: 33.080s


In [3]:
dbsnp_pos=dbsnp_v53.filter_rows(dbsnp_v53.locus.position==1452779)

NameError: name 'dbsnp_v53' is not defined

In [4]:
dbsnp_pos.rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str
chr3:154533782,"[""T"",""C""]","""rs6763896"""
chr7:154533782,"[""G"",""A""]","""rs191411898"""
chrX:154533782,"[""C"",""A""]","""rs202181505"""
chrX:154533782,"[""C"",""G""]","""rs202181505"""
chrX:154533782,"[""C"",""T""]","""rs202181505"""


In [5]:
mt=hl.read_matrix_table(f'{temp_dir}/intervalwgs/WGS_final_march_2020_dbsnp_v53.mt')

In [6]:
mt_chrX=mt.filter_rows(mt.locus.contig=="chrX")

In [7]:
mt_chrX_pos=mt_chrX.filter_rows(mt_chrX.locus.position==154533782)

In [8]:
mt_chrX_pos.rsid.show()

locus,alleles,rsid
locus<GRCh38>,array<str>,str
chrX:154533782,"[""C"",""A""]",NA


Annotate chrX with dbsbp v53

In [9]:
mt_chrX=mt_chrX.annotate_rows(rsidv53=dbsnp_v53.rows()[mt_chrX.row_key].rsid)

In [10]:
mt_chrX_pos=mt_chrX.filter_rows(mt_chrX.locus.position==154533782)

In [11]:
mt_chrX_pos.rsidv53.show()

2020-10-21 14:11:24 Hail: INFO: reading 8 of 212 data partitions


locus,alleles,rsidv53
locus<GRCh38>,array<str>,str
chrX:154533782,"[""C"",""A""]","""rs202181505"""
